In [3]:
!pip install bs4

In [17]:
import bs4
from dotenv import load_dotenv
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings,ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


# 환경변수 가져오기
load_dotenv()

# 문서 로드
loader = WebBaseLoader(
    web_path=("https://n.news.naver.com/article/437/0000416134"),
    bs_kwargs=dict(
        # 특정 요소에서만 파싱하도록 제한하는 필터
        parse_only=bs4.SoupStrainer(
            "div",
            attrs={"class":["newsct_article _article_body","media_end_head go_trans"]}
        )
    )
)

docs = loader.load()

print(docs)

# 문서 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100,chunk_overlap=10)

splits = text_splitter.split_documents(docs)

# 임베딩 생성

embedding = OpenAIEmbeddings()

# 백터스토어 생성
vectorstore = FAISS.from_documents(documents=splits, embedding=embedding)

# 검색기(retriver) 생성
retriver = vectorstore.as_retriever()

# runtime

# 프롬프트
prompt = PromptTemplate.from_template(
"""
당신은 질문-답변을 수행하는 AI 어시스턴트이다.
주어진 문맥에 검색된 다음문맥(context)를 사용해 질문에 답해야 한다.
만약 주어진 문맥에서 답을 찾을 수 없는 경우, 모른다고 이야기하세요. 
한글로 답변해주세요.

#Question:
{question}

#Context:
{context}

#Answer:
"""
)

# LLM 모델
llm = ChatOpenAI(model="gpt-4o", temperature=0)

# Chain 구성
news_chain = (
    {"context":retriver, "question":RunnablePassthrough()}
    |prompt
    |llm
    |StrOutputParser()
)

# 질문
answer = news_chain.invoke("노래 아파트를 부른 사람은 누구야?")

print(answer)

[Document(metadata={'source': 'https://n.news.naver.com/article/437/0000416134'}, page_content='\n\n\n\nJTBC\n\nJTBC\n\n\n\nPICK\n안내\n\n\n언론사가 주요기사로선정한 기사입니다.\n언론사별 바로가기\n닫기\n\n\n\n\n로제 \'아파트\', 빌보드 \'핫100\' 8위 신기록…"꿈 이뤄졌다"\n\n\n\n\n입력2024.10.29. 오전 9:59\n\n기사원문\n \n\n\n\n\n이지현 기자\n\n\n\n\n\n\n\n\n\n\n이지현 기자\n\n구독\n구독중\n\n\n\n\n구독자\n0\n\n\n응원수\n0\n\n\n\n더보기\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n추천\n\n\n\n\n쏠쏠정보\n0\n\n\n\n\n흥미진진\n0\n\n\n\n\n공감백배\n0\n\n\n\n\n분석탁월\n0\n\n\n\n\n후속강추\n0\n\n\n \n\n\n\n댓글\n\n\n\n\n\n본문 요약봇\n\n\n\n본문 요약봇도움말\n자동 추출 기술로 요약된 내용입니다. 요약 기술의 특성상 본문의 주요 내용이 제외될 수 있어, 전체 맥락을 이해하기 위해서는 기사 본문 전체보기를 권장합니다.\n닫기\n\n\n\n\n\n\n\n\n텍스트 음성 변환 서비스 사용하기\n\n\n\n성별\n남성\n여성\n\n\n말하기 속도\n느림\n보통\n빠름\n\n이동 통신망을 이용하여 음성을 재생하면 별도의 데이터 통화료가 부과될 수 있습니다.\n본문듣기 시작\n\n닫기\n\n\n \n\n글자 크기 변경하기\n\n\n\n가1단계\n작게\n\n\n가2단계\n보통\n\n\n가3단계\n크게\n\n\n가4단계\n아주크게\n\n\n가5단계\n최대크게\n\n\n\n\n\n\nSNS 보내기\n\n\n\n인쇄하기\n\n\n\n\n\n\n  \n\n\n\n로제와 브루노 마스가 함께 부른 \'APT.\'(아파트) 〈사진=더블랙레이블〉    블랙핑크 로제와 팝스타 브루노 마스가 함께 부른 \'아파트(A